In [2]:
import polars as pl
from sklearn.ensemble import HistGradientBoostingClassifier
from datetime import datetime
import numpy as np
import geopandas as gpd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from itertools import chain
from functools import partial
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import TimeSeriesSplit, cross_validate
import cat_model_utils as cat_utils

publication strategy is to compare what sorts of models work. To that end we will
Start with a baseline model based on case rates lagged n months XXX
Add in variables and record improvement/change XXXX
correlated municipios
climatic variables (e.g. surface runoff, temperature) XXXX
socioeconomic variables (eg population, income) XXXX
el nino XXXX
land use variables XXXX
do this for n in [1-3]
aggregate to microregions for comparability to other models
may also include auto correlation by
creating autocorrelation vectors with 24 month lag for each municipio
clustering these vectors
creating a categorical one hot variable for cluster membership

### Load all data

In [34]:
##Load categorical land cover data, it is saved in a tricky format so we need to do some processing, converting list of [[band, value]...] to columns of band - proportion of area
band_names = ['water', 'trees', 'grass', 'flooded_vegetation', 'crops', 'shrub_scrub', 'built', 'bare', 'snow_ice']
land_use = (
    pl.read_csv('../data/land_use/dynamic_earth_all_munis_2017.csv')
    .with_columns(
        pl.col('histogram')
        .str.extract_all(
            r'\s[0-9]*\.[0-9]*'
        )
        .list.eval(pl.element().str.strip_chars())
        .cast(pl.List(pl.Float32))
    )
    .with_columns(
        pl.col('histogram').list.sum().alias('total_pixels'),
        pl.col('histogram').list.to_struct(fields=band_names)
    )
    .unnest('histogram')
    .with_columns(
        [pl.col(band)/(pl.col('total_pixels'))for band in band_names],
    )
    .select(pl.exclude('total_pixels'))
    .rename({'CD_MUN':'muni_id'})
)

In [31]:
#Load El nino data, cast and rename for join compatibility
el_nino = pl.read_csv('../data/sst/sst_indices.csv').rename({'YR': 'year', 'MON': 'month'}).with_columns(pl.col('year').cast(str), pl.col('month').cast(pl.UInt32))

In [20]:
#Load monthly dengue case counts -- Note, dengue download script has been updated, you can possibly just run this with a simple read_parquet
#monthly cases are aggregated from the start of the month forward, environmental parameters from the end of the month backwards
monthly_cases = pl.read_parquet('../data/cases/agged/dengue_per_month.parquet').with_columns(pl.col('DT_NOTIFIC').dt.offset_by('1mo').alias('end_date')).with_columns(pl.col('ID_MUNICIP').str.slice(offset=0,length=6).cast(pl.Int64)).sort('DT_NOTIFIC').rename({
'DT_NOTIFIC': 'start_date',
'ID_MUNICIP': 'muni_id'
})

In [21]:
#Load google earth engine dynamic exports
monthly_params = pl.read_parquet('../data/gee_exports_test/all_parameters_2001-01-01_2021-01-01_months.parquet')

In [22]:
#load municipio polygons, used in plotting later
munis = gpd.read_file('../data/brazil/munis/munis_simple.shp').astype({'CD_MUN': 'string'})
munis['CD_MUN'] = munis['CD_MUN'].str.slice(stop=-1).astype(int)

In [35]:
#join datasets
all_data = (
    monthly_cases.join(monthly_params, how='inner', on=['muni_id', 'end_date']).with_columns(pl.col('end_date').alias('month').dt.month())
    .join(el_nino, how='left', on=['year', 'month'])
    .join(land_use, how='left', on='muni_id')
)

all_data

muni_id,start_date,count,x_centroid,y_centroid,NM_MUN,pop,year,cases_per_100k,end_date,temporal_sdm_both,start_date_right,temporal_sdm_albopictus,start_date_temporal_sdm_albopictus,temporal_sdm_aegypti,start_date_temporal_sdm_aegypti,EVI,start_date_EVI,total_evaporation_sum,start_date_total_evaporation_sum,total_precipitation_sum,start_date_total_precipitation_sum,evaporation_from_open_water_surfaces_excluding_oceans_min,start_date_evaporation_from_open_water_surfaces_excluding_oceans_min,soil_temperature_level_1,start_date_soil_temperature_level_1,runoff_sum,start_date_runoff_sum,surface_runoff_min,start_date_surface_runoff_min,volumetric_soil_water_layer_1,start_date_volumetric_soil_water_layer_1,dewpoint_temperature_2m,start_date_dewpoint_temperature_2m,temperature_2m,start_date_temperature_2m,total_precipitation_min,start_date_total_precipitation_min,month,NINO1+2,ANOM,NINO3,ANOM_1,NINO4,ANOM_2,NINO3.4,ANOM_3,water,trees,grass,flooded_vegetation,crops,shrub_scrub,built,bare,snow_ice
i64,date,u32,f64,f64,str,i64,str,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,f64,date,u32,f64,f64,f64,f64,f64,f64,f64,f64,f32,f32,f32,f32,f32,f32,f32,f32,f32
261090,2001-01-01,4,6.9004e6,9.0303e6,"""Pesqueira""",57600,"""2001""",6.944444,2001-02-01,0.815822,2000-12-03,0.889802,2000-12-03,0.848074,2000-12-03,0.263019,2001-01-01,-0.000895,2001-01-01,0.000085,2001-01-01,-0.000351,2001-01-01,301.865679,2001-01-01,3.5763e-7,2001-01-01,3.4273e-7,2001-01-01,0.069654,2001-01-01,288.65525,2001-01-01,296.732309,2001-01-01,0.000085,2001-01-01,2,25.71,-0.38,26.17,-0.21,27.1,-0.99,26.18,-0.56,0.001442,0.054273,0.003199,0.000601,0.043381,0.845582,0.011995,0.038997,0.000531
261160,2001-01-01,93,7.1010e6,9.0621e6,"""Recife""",1437190,"""2001""",6.470961,2001-02-01,0.147099,2000-12-03,0.079524,2000-12-03,0.124176,2000-12-03,0.206202,2001-01-01,-0.002609,2001-01-01,0.000917,2001-01-01,-0.001167,2001-01-01,301.231897,2001-01-01,0.000095,2001-01-01,0.000007,2001-01-01,0.243936,2001-01-01,293.678382,2001-01-01,299.746231,2001-01-01,0.000917,2001-01-01,2,25.71,-0.38,26.17,-0.21,27.1,-0.99,26.18,-0.56,0.058746,0.132567,0.01401,0.00674,0.068523,0.008962,0.698849,0.010218,0.001385
221140,2001-01-01,14,6.3021e6,9.2600e6,"""Várzea Grande""",4480,"""2001""",312.5,2001-02-01,0.304474,2000-12-03,0.113402,2000-12-03,0.27118,2000-12-03,0.361755,2001-01-01,-0.003763,2001-01-01,0.003502,2001-01-01,-0.000998,2001-01-01,299.873278,2001-01-01,0.000029,2001-01-01,0.000026,2001-01-01,0.33123,2001-01-01,294.040172,2001-01-01,298.064608,2001-01-01,0.003502,2001-01-01,2,25.71,-0.38,26.17,-0.21,27.1,-0.99,26.18,-0.56,0.0,0.431499,0.175103,0.008541,0.056766,0.313681,0.012977,0.001391,0.000042
330250,2001-01-01,48,6.1183e6,7.4575e6,"""Magé""",210861,"""2001""",22.763811,2001-02-01,0.397118,2000-12-03,0.38288,2000-12-03,0.360123,2000-12-03,0.362983,2001-01-01,-0.005113,2001-01-01,0.003515,2001-01-01,-0.000491,2001-01-01,300.473692,2001-01-01,0.000505,2001-01-01,0.000063,2001-01-01,0.381232,2001-01-01,294.807948,2001-01-01,299.037432,2001-01-01,0.003515,2001-01-01,2,25.71,-0.38,26.17,-0.21,27.1,-0.99,26.18,-0.56,0.005546,0.714475,0.071664,0.002938,0.015708,0.018429,0.167235,0.002981,0.001025
312770,2001-01-01,1747,6.2680e6,7.8794e6,"""Governador Val…",248960,"""2001""",701.719152,2001-02-01,0.10519,2000-12-03,0.042876,2000-12-03,0.083847,2000-12-03,0.276075,2001-01-01,-0.004198,2001-01-01,0.000386,2001-01-01,-0.000926,2001-01-01,299.908398,2001-01-01,0.000441,2001-01-01,0.000005,2001-01-01,0.360168,2001-01-01,292.610876,2001-01-01,298.945548,2001-01-01,0.000386,2001-01-01,2,25.71,-0.38,26.17,-0.21,27.1,-0.99,26.18,-0.56,0.011376,0.296313,0.123885,0.008196,0.015853,0.499543,0.024559,0.018607,0.001668
313580,2001-01-01,50,6.3789e6,8.1435e6,"""Jequitinhonha""",22869,"""2001""",218.636582,2001-02-01,0.454964,2000-12-03,0.336327,2000-12-03,0.432499,2000-12-03,0.446538,2001-01-01,-0.002907,2001-01-01,0.000233,2001-0

## Experiments

In [ ]:
#Dictionary to hold all results
ALL_RESULTS = {}

##### 1. Ridge Regression Classification using Binary/Ternary and Simple/Relative - Most basic classification model (linear least squares regression)

##### 2. Classification using Gradient Boosting Classifier

##### 3. Time Lag Search

##### 4. Non-contiguous time-lag search

##### 5. Environmental feature search

##### 6. Lag-correlated municipios